### 🧱 Step 1: Audio I/O (Hear & Speak Back)

In [1]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav

In [2]:
# Settings
duration = 3   # seconds
fs = 16000     # sample rate (16 kHz)

In [5]:
print("🎤 Recording... Speak now!")
recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
sd.wait()  # wait until done
print("✅ Recording complete!")

# Save as a file (optional)
wav.write("Recordings/whispy_test.wav", fs, recording)

# Playback
print("🔊 Playing back your voice...")
sd.play(recording, fs)
sd.wait()
print("🎉 Done! That’s your voice.")


🎤 Recording... Speak now!
✅ Recording complete!
🔊 Playing back your voice...
🎉 Done! That’s your voice.


### 🧱 Step 2: Whispy learns to understand words (STT)

In [8]:
import sounddevice as sd
import queue
import json
from vosk import Model, KaldiRecognizer

In [ ]:
# Load Vosk model
model = Model("model")
recognizer = KaldiRecognizer(model, 16000)

In [9]:
q = queue.Queue()

In [ ]:
def callback(indata, frames, time, status):
    if status:
        print(status)
    q.put(bytes(indata))

In [13]:
# Open microphone stream
with sd.RawInputStream(samplerate=16000, blocksize=8000, dtype='int16',
                       channels=1, callback=callback):

    print("🎤 Whispy is listening... Say something! (say 'stop' to quit)")

    while True:
        data = q.get()
        if recognizer.AcceptWaveform(data):
            result = json.loads(recognizer.Result())
            text = result["text"]
            if text:
                print("📝 Whispy heard:", text)
                if "stop" in text.lower():
                    print("👋 Whispy says: Bye Mansour!")
                    break

🎤 Whispy is listening... Say something! (say 'stop' to quit)
📝 Whispy heard: comedy respite how are your derby
📝 Whispy heard: they know i know how are you speak know how it then and of a lot of
📝 Whispy heard: yeah
📝 Whispy heard: as to come across a halfwit name of the and opponents of the colonel to
📝 Whispy heard: but
📝 Whispy heard: we were looking with well florida madame leash
📝 Whispy heard: isn't that available
📝 Whispy heard: is
📝 Whispy heard: the media to stop
👋 Whispy says: Bye Mansour!
